In [1]:
!pip install openpyxl
!pip install xlwings


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Checking Excel sheets

In [3]:
assignment_number = "Assignment_1"

assignment1_folder_path = "/content/drive/MyDrive/Classroom/Technology For Financial Forecasting 3BBAF&E B BRC 109/Assignment_1"

assignment2_excel_answer_key_path = "/content/drive/MyDrive/Classroom/Technology For Financial Forecasting 3BBAF&E B BRC 109/Assignments Files/Assignment_ sol 1.xlsx"


In [4]:
import openpyxl
import os
import pandas as pd

def write_to_file(string_data, file_path=f"logfile_{assignment_number}.txt"):
  file_path = os.path.join(os.getcwd(), file_path)
  # print('write to file is executed', file_path)
  with open(file_path, 'a+') as file:
    file.writelines(string_data)


def get_all_excel_filenames(folder_path):
    files = os.listdir(folder_path)
    excel_files = []
    for f in files:
        if f.endswith('.xlsx'):
            excel_files.append(f)
    write_to_file(f"Found {len(excel_files)} excel files to evaluate. \n\n\n\n ")
    return excel_files


def check_value(cell_address, worksheet, log=True):
    if log:
        write_to_file(f" \n {worksheet[cell_address].value} \n ")
    return worksheet[cell_address].value


def check_data_type(cell_address, worksheet):
    write_to_file(f" \n {worksheet[cell_address].data_type} (__data_type) \n ")
    return worksheet[cell_address].data_type


def empty_data_structure():
    data_structure = dict()
    data_structure['Name'] = None
    data_structure['Roll Number'] = None
    data_structure['Result'] = 0
    data_structure['Error Sum'] = 0
    data_structure['Value Sum'] = 0
    data_structure['Formula Sum'] = 0

    return data_structure

try:
    import xlwings as xw
    xlwings_available = True
except ImportError:
    xlwings_available = False

def get_formula_multiengine(file_path, sheet_name, cell_address):
    # Try openpyxl first
    try:
        wb = openpyxl.load_workbook(file_path)
        ws = wb[sheet_name] if sheet_name in wb.sheetnames else wb.active
        cell = ws[cell_address]

        # Case 1: Cell has .formula attribute (rare)
        if hasattr(cell, "formula") and isinstance(cell.formula, str):
            return cell.formula, "openpyxl (cell.formula)"

        # Case 2: cell.value is a string starting with =
        if isinstance(cell.value, str) and cell.value.startswith("="):
            return cell.value, "openpyxl (cell.value)"

        # Case 3: cell.value is an ArrayFormula or object, reject and fall back
        if hasattr(cell.value, 'text'):
            return cell.value.text, "openpyxl (ArrayFormula.text)"

        # Case 4: Unusable object (e.g., openpyxl ArrayFormula), fall through to xlwings
    except Exception as e:
        # Continue to xlwings
        pass

    # Try xlwings if openpyxl didn't return usable formula
    if xlwings_available:
        try:
            app = xw.App(visible=False)
            wb = app.books.open(file_path)
            sht = wb.sheets[sheet_name] if sheet_name in [s.name for s in wb.sheets] else wb.sheets[0]
            formula = sht.range(cell_address).formula
            wb.close()
            app.quit()
            return formula, "xlwings"
        except Exception as e:
            return None, f"xlwings error: {str(e)}"

    return None, "No formula found"




def check_specific_answers(empty_data, column_name, row_number, number_of_question, formula_worksheet, value_worksheet,
                           values_answer_key, formula_answer_key, file_path):
    values_answer_key = values_answer_key or {}
    formula_answer_key = formula_answer_key or {}

    for i in range(number_of_question):
        write_to_file(f"{'-'* 4} \n \n Checking question number: {i+1}")

        cell_address = column_name + str(row_number + i)
        write_to_file(f"        checking cell:     [{cell_address}]  \n ")

        write_to_file(f"Checking formula output for any errors  \n ")
        value = check_value(cell_address, value_worksheet, False)
        is_error_value = value in (
            "#N/A", "#DIV/0!", "#NAME?", "#NULL!", "#NUM!", "#REF!", "#VALUE!", "#####", "Circular Reference"
        )

        if is_error_value:
            write_to_file(
                f"Error detected in value: >>{value}<<  \n "
            )
            empty_data['Error Sum'] += 1
            write_to_file(f"Your ERROR count increased: {empty_data['Error Sum']}  \n ")

        # Proceed with value checking anyway
        if values_answer_key is not None and cell_address in values_answer_key:
            if checking_if_values_are_correct(value, values_answer_key[cell_address]):
                empty_data['Value Sum'] += 1
                write_to_file(f"Your VALUE count increased: {empty_data['Value Sum']}  \n ")
            else:
                write_to_file(
                    f"   Warning: Output {value} of {type(value)} must be as specified in assignment : {values_answer_key[cell_address]}  of {type(values_answer_key[cell_address])}. \n "
                )
        else:
            write_to_file(f"          No output compulsion for this cell. \n ")

        # Proceed with formula checking regardless
        if formula_answer_key is not None and cell_address in formula_answer_key:
            formula_string, engine_used = get_formula_multiengine(file_path=file_path, sheet_name=formula_worksheet.title, cell_address=cell_address)

            try:
                if hasattr(formula_string, 'text'):
                    formula_string = formula_string.text
                elif not isinstance(formula_string, str):
                    formula_string = str(formula_string)
            except Exception as e:
                write_to_file(f"Error parsing formula: {e}\n")

            if not formula_string or not isinstance(formula_string, str):
                write_to_file("Could not retrieve usable formula from any engine.\n")
            else:
                write_to_file(f"Formula found using {engine_used}: {formula_string}\n")

                expected_results = formula_answer_key[cell_address]
                if isinstance(expected_results, str):
                    expected_results = [expected_results]

                formula_upper = formula_string.upper()

                if any(func.upper() in formula_upper for func in expected_results):
                    empty_data['Formula Sum'] += 1
                    write_to_file(f"Your FORMULA count increased: {empty_data['Formula Sum']}  \n ")
                else:
                    write_to_file(
                        f"   Warning: You should have used one of: {', '.join(expected_results)} but you used: {formula_string}.\n "
                    )
        else:
            write_to_file(f"          No formula compulsion for this cell. \n ")

        if cell_address in values_answer_key or cell_address in formula_answer_key:
            empty_data['Result'] += 1
            write_to_file(f"#####Your RESULT count increased: {empty_data['Result']} ###### \n ")


    return empty_data

def checking_if_values_are_correct(returned_value, actual_value):
    if returned_value == None or actual_value == None:
      return False

    if returned_value == actual_value:
        return True

    try:
        temp_returned_value = float(returned_value)
        temp_actual_value = float(actual_value)

        if temp_returned_value == temp_actual_value:
            return True
    except ValueError:
        pass

    if isinstance(actual_value, list):
      if returned_value in actual_value:
        return True

    if returned_value in ("True", "TRUE", "true", True) and actual_value in ("True", "TRUE", "true", True):
        return True

    elif returned_value in ("False", "FALSE", "false", False) and actual_value in ("False", "FALSE", "false", False):
        return True

    else:
        return False


def evaluate_excel_file(file_path, column_name, row_number, number_of_question, values_answer_key, formula_answer_key,
                        worksheet_name='Answers', student_name_cell='B2', student_roll_no_cell='B3'):
    write_to_file(f"{'~' * 80} \n\n \n ")

    if file_path:
        file_name = file_path.split('/')[-1]
        write_to_file(f" \n \n {'+'* 25} \n Working with {file_name}   \n \n")
    else:
        write_to_file("No file_path  \n ")
        return

    try:
        formula_wb = openpyxl.load_workbook(file_path)
    except:
        write_to_file(f"Could not find the {file_path}  \n ")
        return

    if worksheet_name not in formula_wb.sheetnames:
        new_worksheet = formula_wb.sheetnames[0]
        write_to_file(f"Since '{worksheet_name}' is not there. This tools is using {new_worksheet}  \n ")
        worksheet_name = new_worksheet
    try:
        formula_worksheet = formula_wb[worksheet_name]
    except KeyError:
        write_to_file("sheet named 'Answers' must be there and contain answers.  \n ")
        return

    try:
        value_wb = openpyxl.load_workbook(file_path, data_only=True)
    except:
        write_to_file(f"Could not find the {file_path}  \n ")
        return

    try:
        value_worksheet = value_wb[worksheet_name]
    except KeyError:
        write_to_file("sheet named 'Answers' must be there and contain answers.  \n ")
        return

    try:
        name = check_value(student_name_cell, formula_worksheet)
    except:
        write_to_file(f'Could not find your official Name in B1.  \n ')
        return

    try:
        roll_no = check_value(student_roll_no_cell, formula_worksheet)
    except:
        write_to_file(f'Could not find your roll number in B2.  \n ')
        return

    result_data = check_specific_answers(empty_data_structure(), column_name, row_number, number_of_question,
                                     formula_worksheet, value_worksheet, values_answer_key, formula_answer_key, file_path)

    result_data['Name'] = name
    result_data['Roll Number'] = roll_no

    write_to_file(str(result_data))
    write_to_file(f"{'*' * 80} \n\n \n ")

    return result_data


class CheckExcelFiles:
    def __init__(self, folder_path, formula_answer_key={}, values_answer_key={}):
        self.values_answer_key = values_answer_key
        self.path = folder_path
        self.formula_answer_key = formula_answer_key


    def evaluate_all_excel_files(self, column_name, row_number, number_of_question):
        result_data_list = []
        excel_filenames = get_all_excel_filenames(self.path)

        for filename in excel_filenames:

            file_path = os.path.join(self.path, filename)
            print(file_path)
            data = evaluate_excel_file(file_path, column_name, row_number, number_of_question, self.values_answer_key,
                                       self.formula_answer_key)
            print(data)
            if data:
                result_data_list.append(data)
        df = pd.DataFrame.from_records(result_data_list)

        return df



In [5]:
def create_answer_key(excel_file_path, column_name, row_number, number_of_question, worksheet_name = 'Answers'):
  formula_wb = openpyxl.load_workbook(excel_file_path)
  formula_ws = formula_wb[worksheet_name]

  value_wb = openpyxl.load_workbook(excel_file_path, data_only=True)
  value_ws = value_wb[worksheet_name]

  formula_answer_key = create_formula_answers_loop(column_name, row_number, number_of_question, formula_ws)
  value_answer_key = create_answers_loop(column_name, row_number, number_of_question, value_ws)

  return formula_answer_key, value_answer_key

In [6]:
def create_answers_loop(column_name, row_number, number_of_question, worksheet):
  result = dict()
  for i in range(number_of_question):
    cell_address = column_name + str(row_number+i)
    result[cell_address] = check_value(cell_address, worksheet)

  return result

In [7]:
def create_formula_answers_loop(column_name, row_number, number_of_question, worksheet):
  result = dict()
  for i in range(number_of_question):
    cell_address = column_name + str(row_number+i)
    print(cell_address)
    cell_value = check_value(cell_address, worksheet)
    print(cell_value)
    # Check if the cell value is an ArrayFormula object and get its text
    if isinstance(cell_value, openpyxl.worksheet.formula.ArrayFormula):
        cell_value = cell_value.text
    result[cell_address] = str(cell_value).split("(")[0]

  return result

### **Instructions for Using the Tool**

**Step 1:** Provide the folder path where your solution Excel file is located.

**Step 2:** Mention the starting cell from where the answers begin in that Excel file. The evaluate_excel_file function tells which cell contain student_name_cell='B2', student_roll_no_cell='B3'

---

### **Example for Better Understanding**

* In the next input cell, enter the folder path where your solution Excel file is stored.
* The tool will look for answers starting from **cell B5** to **B14**, meaning a total of **10 questions**.
* Ensure that your answers are placed in **column B**, from row 5 to row 14.
* The tool will specifically check the sheet named **"Sheet1"** in your Excel file.

---


In [8]:
# creating the answer key from answer excel sheet
formula_answer, value_answer = create_answer_key(assignment2_excel_answer_key_path, 'B', 5, 10, "Sheet1")

B5
=COUNTIF(G3:G12,"North")
B6
=SUMIF(J3:J12,"Electronics",H3:H12)
B7
=AVERAGE(I3:I12)
B8
=MAX(H3:H12)
B9
=COUNTIF(I3:I12,"<0")
B10
=PERCENTILE(H3:H12,0.75)
B11
=SUMIF(G3:G12,"East",I3:I12)
B12
=QUARTILE(H3:H12,3) - QUARTILE(H3:H12,1)
B13
=COUNTIF(K3:K12, ">" & DATE(2024,1,7))
B14
=SUMIF(J3:J12,"Furniture",I3:I12)


In [9]:
formula_answer

{'B5': '=COUNTIF',
 'B6': '=SUMIF',
 'B7': '=AVERAGE',
 'B8': '=MAX',
 'B9': '=COUNTIF',
 'B10': '=PERCENTILE',
 'B11': '=SUMIF',
 'B12': '=QUARTILE',
 'B13': '=COUNTIF',
 'B14': '=SUMIF'}

In [10]:
value_answer

{'B5': 3,
 'B6': 97000,
 'B7': 2120,
 'B8': 30000,
 'B9': 2,
 'B10': 24250,
 'B11': -2500,
 'B12': 8750,
 'B13': 3,
 'B14': 5000}

In [11]:
values = ['IFERROR',
  'MATCH',
  'COUNTIF',
  'IFERROR',
  'INDEX',
  'COUNTIFS',
  'XLOOKUP',
  'SUMIFS',
  'INDEX', 'FILTER', 'MAXIFS', 'SUMIF', 'AVERAGEIFS', 'MINIFS', 'VLOOKUP', 'AVERAGEIF', 'MAX', 'AVERAGE', 'PERCENTILE', 'QUARTILE', ]

formula_answer = {
    key: list(values)
    for key in formula_answer
}


# Step 3
You have to give the path of folder which consist students excel files. It will check from B5 till 10 questions for each students.

In [ ]:

checker_object = CheckExcelFiles(assignment1_folder_path, formula_answer, value_answer)

result = checker_object.evaluate_all_excel_files('B',  5, 10)


/content/drive/MyDrive/Classroom/Technology For Financial Forecasting 3BBAF&E B BRC 109/Assignment_1/Assignment_1 name roll no (25).xlsx
{'Name': 'Manansh Vyas ', 'Roll Number': 2423135.0, 'Result': 10, 'Error Sum': 1, 'Value Sum': 5, 'Formula Sum': 9}
/content/drive/MyDrive/Classroom/Technology For Financial Forecasting 3BBAF&E B BRC 109/Assignment_1/Assignment_1 name roll no (24).xlsx
{'Name': None, 'Roll Number': None, 'Result': 10, 'Error Sum': 0, 'Value Sum': 0, 'Formula Sum': 0}
/content/drive/MyDrive/Classroom/Technology For Financial Forecasting 3BBAF&E B BRC 109/Assignment_1/Assignment_1 name roll no (23).xlsx
{'Name': 'Sameeksha Sathish ', 'Roll Number': 2423148, 'Result': 10, 'Error Sum': 0, 'Value Sum': 9, 'Formula Sum': 10}
/content/drive/MyDrive/Classroom/Technology For Financial Forecasting 3BBAF&E B BRC 109/Assignment_1/Assignment_1 name roll no (22).xlsx
{'Name': 'Ritika Sahay', 'Roll Number': 2423163, 'Result': 10, 'Error Sum': 0, 'Value Sum': 9, 'Formula Sum': 10}
/c

In [ ]:
result['Name'] = result['Name'].apply(lambda x: str(x).strip().title())

In [ ]:
new_df = result.sort_values(by='Formula Sum', ascending= False)

# new_df.to_csv("Assignment_1 Metrics", index = False)
clean_df = new_df.drop_duplicates(subset=['Roll Number'])
clean_df.to_csv(f"{assignment_number} Metrics.csv", index = False)

clean_df

,Name,Roll Number,Result,Error Sum,Value Sum,Formula Sum
3,Ritika Sahay,2423163.0,10,0,9,10
2,Sameeksha Sathish,2423148.0,10,0,9,10
4,Abhir Deepak,2423103.0,10,0,8,10
5,Kshitij H,2423130.0,10,0,8,10
7,None,NaN,10,0,6,10
6,Stuti Sinha,2423156.0,10,0,9,10
15,Aanchal,2423102.0,10,0,7,10
8,Jiya Mehta,2423121.0,10,0,7,10
10,Gaybraham Variability,2423104.0,10,0,8,10
11,Rudra Sharma,2423145.0,10,0,7,10


# EOF